# SQL

In [51]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [216]:
n_topics = 10

In [5]:
#In Python: Define a database name, and your username for your computer. 
dbname = 'oped_db'
username = 'varun'

In [6]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url

postgres://varun@localhost/oped_db


In [7]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [217]:
# load a database from CSV
dataall = pd.DataFrame.from_csv('dataall.csv')
datadate = pd.DataFrame.from_csv('datadate.csv')
topicweights = pd.DataFrame.from_csv('topicweights{0}.csv'.format(n_topics))

In [ ]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
## df is any pandas dataframe 
dataall.to_sql('orig', engine, if_exists='replace')
datadate.to_sql('dates_and_tidy', engine, if_exists='replace')
topicweights.to_sql('topic_weights', engine, if_exists='replace')

In [11]:
## Now try the same queries, but in python!
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username)

In [28]:
# query:
sql_query = """
SELECT topic_weights.index, topic_weights.topic0 FROM topic_weights
WHERE topic0 > 0.2;
"""
data_from_sql = pd.read_sql_query(sql_query,con)
data_from_sql.head()

,index,topic0
0,4,0.348150
1,34,0.364297
2,141,0.337865
3,281,0.279798
4,287,0.470296


In [75]:
# query:
def query_for_person(firstname,lastname):
    return """
    SELECT orig.share_count, topic_weights.*
    FROM orig
        JOIN topic_weights
            ON orig.index = topic_weights.index
    WHERE orig.first_name='{0}' AND orig.last_name='{1}';
    """.format(firstname,lastname)

In [76]:
sql_query = query_for_person('paul','krugman')
data_from_sql = pd.read_sql_query(sql_query,con)
data_from_sql.head()

,share_count,index,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,...,topic40,topic41,topic42,topic43,topic44,topic45,topic46,topic47,topic48,topic49
0,818,38,0.000061,0.000061,0.000061,0.000061,0.000061,0.225027,0.000061,0.055129,...,0.000061,0.000061,0.000061,0.000061,0.000061,0.000061,0.000061,0.000061,0.000061,0.043902
1,790,72,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067,...,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067,0.020350,0.020271,0.044775
2,4503,152,0.000064,0.024821,0.000064,0.000064,0.000064,0.000064,0.000064,0.000064,...,0.024696,0.000064,0.000064,0.000064,0.000064,0.073259,0.000064,0.000064,0.067449,0.201081
3,1590,121,0.000064,0.000064,0.045788,0.000064,0.000064,0.000064,0.000064,0.000064,...,0.000064,0.000064,0.000064,0.000064,0.000064,0.000064,0.000064,0.035754,0.000064,0.068667
4,6806,202,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.023220,...,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.122524


In [77]:
data_from_sql['log_share_count'] = np.log10(data_from_sql['share_count'])

In [92]:
xmin = np.min(data_from_sql['log_share_count'])
xmax = np.max(data_from_sql['log_share_count'])
bins = np.linspace(xmin,xmax,10)
binnum = np.digitize(data_from_sql['log_share_count'],bins)
binnum[binnum <= 5] = 0
binnum[binnum > 5] = 1
data_from_sql['bin_share_count'] = binnum

In [78]:
plt.hist(data_from_sql['log_share_count'])
plt.show()

# Machine Learning

In [66]:
from sklearn import linear_model, neighbors

In [133]:
n_topics = 50
featurenames = ['topic{0}'.format(i) for i in range(n_topics)]
features = data_from_sql[featurenames]
viralityname = 'log_share_count'
virality = data_from_sql[viralityname]

In [181]:
def split_data(data,frac=0.7):
    datanew = data.sample(frac=1)
    nrows = len(datanew)
    idx = int(nrows*frac)
    return {'train': datanew.iloc[:idx], 'test': datanew.iloc[idx:]}
    # use 70-30 split

In [99]:
def train_model(split_data,model,errorfun,**kwargs):
    datatrain = split_data['train']
    datatest = split_data['test']
    model.fit(datatrain[featurenames],datatrain[viralityname],**kwargs)
    train_pred = model.predict(datatrain[featurenames])
    test_pred = model.predict(datatest[featurenames])
    train_error = errorfun(train_pred,datatrain[viralityname])
    test_error = errorfun(test_pred,datatest[viralityname])
    print('Training error: {0}'.format(train_error))
    print('Test error: {0}'.format(test_error))

In [214]:
datasplit = split_data(data_from_sql)

## Classification

In [117]:
def model_error_class(predicted,actual):
    nobs = actual.size
    return 1.0/nobs*np.sum((actual - predicted)**2)

### Logistic Regression

In [123]:
fig = plt.figure(1)
ax = fig.gca()
ax = ax.matshow(data_from_sql[featurenames], cmap=plt.cm.gray)
plt.show()

In [113]:
model = linear_model.LogisticRegression()
model.fit(datatrain[featurenames],datatrain['bin_share_count'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### K-Nearest Neighbors

In [ ]:
model = neighbors.KNeighbors(n_neighbors=i)
train_model(datasplit,model)

## Regression

In [172]:
def model_error_regr(predicted,actual):
    nobs = actual.size
    return np.sqrt(1.0/(2.0*nobs)*np.sum((actual - predicted)**2))

In [173]:
def coeff_regr(predicted,actual):
    avg = np.mean(actual)
    sstot = np.sum((actual - avg)**2)
    ssres = np.sum((actual - predicted)**2)
    return 1 - ssres/sstot

### Linear Regression

In [193]:
model = linear_model.LinearRegression()

In [194]:
train_model(datasplit,model,coeff_regr)

Training error: 0.438524378017
Test error: -4.37725989263


In [139]:
actual = data_from_sql[viralityname]
pred = model.predict(data_from_sql[featurenames])

In [143]:
plt.plot(actual,pred,'o')
plt.show()

In [195]:
model = neighbors.KNeighborsRegressor(n_neighbors=10)

In [196]:
train_model(datasplit,model,coeff_regr)

Training error: 0.0930888034172
Test error: 0.182169596621


In [198]:
actual = data_from_sql[viralityname]
pred = model.predict(data_from_sql[featurenames])

In [199]:
plt.plot(actual,pred,'o')
plt.show()

In [202]:
train_model(datasplit,model,coeff_regr)

Training error: 0.123736648092
Test error: 0.159580338504


## K-Nearest Neighbors

In [ ]:
np.mean(dataallnew[viralityname])

In [215]:
for i in range(1,20):
    model = neighbors.KNeighborsRegressor(n_neighbors=i)
    train_model(datasplit,model,coeff_regr)

Training error: 1.0
Test error: -1.57570050605
Training error: 0.486015701869
Test error: -0.498145944473
Training error: 0.376983043397
Test error: -0.515264379594
Training error: 0.313442065944
Test error: -0.503127637623
Training error: 0.30653086839
Test error: -0.462875305515
Training error: 0.314335191992
Test error: -0.344951428193
Training error: 0.306781189054
Test error: -0.386864302163
Training error: 0.262677780013
Test error: -0.250925258364
Training error: 0.250909172593
Test error: -0.212311465435
Training error: 0.242121230302
Test error: -0.140216610601
Training error: 0.235558480477
Test error: -0.150352326529
Training error: 0.225711968947
Test error: -0.0702849199616
Training error: 0.234862458749
Test error: -0.0463540656595
Training error: 0.229635379158
Test error: -0.0525599811083
Training error: 0.240268764203
Test error: -0.0395068259081
Training error: 0.240871267392
Test error: -0.0299143932902
Training error: 0.24218076312
Test error: -0.0118178267174
Train